In [ ]:
import boto3
import json
import decimal
from boto3.dynamodb.conditions import Key, Attr

In [ ]:
# Identify the profile to use for this session
session = boto3.Session(profile_name='myadmin')

In [ ]:
dynamodb = session.resource('dynamodb', region_name='us-west-2')

In [ ]:
def create_movie_table():
    table = dynamodb.create_table(
    TableName='Movies',
    KeySchema=[
        {
            'AttributeName': 'year',
            'KeyType': 'HASH'  #Partition key
        },
        {
            'AttributeName': 'title',
            'KeyType': 'RANGE'  #Sort key
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'year',
            'AttributeType': 'N'
        },
        {
            'AttributeName': 'title',
            'AttributeType': 'S'
        },

    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    }
    )

In [ ]:
create_movie_table()

In [ ]:
def load_movie_data(movie_data_location):
    table = dynamodb.Table('Movies')
    
    how_many_to_load = 100
    
    with open(movie_data_location) as json_file:
        movies = json.load(json_file, parse_float = decimal.Decimal)
        for movie in movies[:how_many_to_load]:
            
            year = int(movie['year'])
            title = movie['title']
            info = movie['info']

            print("Adding movie:", year, title)

            table.put_item(
               Item={
                   'year': year,
                   'title': title,
                   'info': info,
                }
            )

In [ ]:
movie_data_location = r'C:\AWSCertificationCourse\Demo\CloudDatabaseServices\movie_data\moviedata.json'
load_movie_data(movie_data_location)

In [ ]:
# Helper class to convert a DynamoDB item to JSON.
class DecimalEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, decimal.Decimal):
            if o % 1 > 0:
                return float(o)
            else:
                return int(o)
        return super(DecimalEncoder, self).default(o)

In [ ]:
def get_movie(year, title):
    table = dynamodb.Table('Movies')
    
    try:
        response = table.get_item(
            Key={
                'year': year,
                'title': title
            }
        )
    except ClientError as e:
        print(e.response['Error']['Message'])
    else:        
        if 'Item' in response:
            item = response['Item']
            print("GetItem succeeded:")
            print(json.dumps(item, indent=4, cls=DecimalEncoder))
        else:
            print('Movie not found')
    

In [ ]:
get_movie(year=2013,title='Rush')

In [ ]:
def query_movies(year):
    table = dynamodb.Table('Movies')

    print("Movies from {0}".format(year))

    response = table.query(
        KeyConditionExpression=Key('year').eq(year)
    )

    for i in response['Items']:
        print(i['year'], ":", i['title'])

In [ ]:
query_movies(2012)

In [ ]:
def query_movies_by_year_title_prefix(year, start_letter, end_letter):
    table = dynamodb.Table('Movies')

    print("Movies from {0} - titles {1}-{2}, with genres and lead actor".format(year, start_letter, end_letter))

    response = table.query(
        ProjectionExpression="#yr, title, info.genres, info.actors[0]",
        ExpressionAttributeNames={ "#yr": "year" }, # Expression Attribute Names for Projection Expression only.
        KeyConditionExpression=Key('year').eq(year) & Key('title').between(start_letter, end_letter)
    )

    for i in response[u'Items']:
        print(json.dumps(i, indent=4, cls=DecimalEncoder))

In [ ]:
query_movies_by_year_title_prefix(2013, 'A','D')

In [ ]:
table = dynamodb.Table('Movies')

In [ ]:
# Add one movie
title = "The Big New Movie"
year = 2015

response = table.put_item(
   Item={
        'year': year,
        'title': title,
        'info': {
            'plot':"Nothing happens at all.",
            'rating': decimal.Decimal(0)
        }
    }
)

print("PutItem succeeded:")
print(json.dumps(response, indent=4, cls=DecimalEncoder))

In [ ]:
get_movie(year=2015, title= "The Big New Movie")

In [ ]:
# Update one movie
title = "The Big New Movie"
year = 2015

response = table.update_item(
    Key={
        'year': year,
        'title': title
    },
    UpdateExpression="set info.rating = :r, info.plot=:p, info.actors=:a",
    ExpressionAttributeValues={
        ':r': decimal.Decimal(8.5),
        ':p': "Everything happens all at once.",
        ':a': ["Larry", "Moe", "Curly"]
    },
    ReturnValues="UPDATED_NEW"
)

print("UpdateItem succeeded:")
print(json.dumps(response, indent=4, cls=DecimalEncoder))    

In [ ]:
# Increment Counter
title = "The Big New Movie"
year = 2015

response = table.update_item(
    Key={
        'year': year,
        'title': title
    },
    UpdateExpression="set info.rating = info.rating + :val",
    ExpressionAttributeValues={
        ':val': decimal.Decimal(1)
    },
    ReturnValues="UPDATED_NEW"
)

print("UpdateItem succeeded:")
print(json.dumps(response, indent=4, cls=DecimalEncoder))

In [ ]:
get_movie(year=2015, title= "The Big New Movie")

In [ ]:
# Delete a movie
title = "The Big New Movie"
year = 2015

response = table.delete_item(
    Key={
        'year': year,
        'title': title
    }
)

print("UpdateItem succeeded:")
print(json.dumps(response, indent=4, cls=DecimalEncoder))    